hi bye

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
import pandas as pd
import numpy as np
!pip install mne
import mne 

In [ ]:
#### Load data ####
raw_data = {"nabeha":[], "heidi":[], "avni":[]}

# Iterate thru all trials for each subject
for subj in raw_data.keys():
  for i in range(1,6):
    # Load data from CSV into an array
    trial_data = np.genfromtxt('/content/drive/Shareddrives/Neuromancers_Data/'+subj+'_data/OpenBCISession_'+subj+'_'+str(i)+'/BrainFlow-RAW_'+subj+'_'+str(i)+'_0.csv', delimiter='\t', dtype=str)
    trial_data = np.char.replace(trial_data, '\t', ' ')
    trial_data = trial_data.astype(float)

    # Declares channel names and types of each set of data
    ch_names = ['Channel {}'.format(i) for i in range(trial_data.shape[1])]
    ch_types = ['eeg' for i in range(trial_data.shape[1])]

    # Create info structures and RawArray objects for each set of data
    sfreq = 250  # sample rate in Hz
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    raw_array = mne.io.RawArray(trial_data.T, info)

    # Removing irrelevant channels
    ch_names = [raw_array.ch_names]
    ch_names_to_keep = [ch_names[0][0:10]]
    raw_array = raw_array.pick_channels(ch_names_to_keep[0])

    # Add RawArray
    raw_data[subj].append(raw_array)

In [ ]:
#### Truncate and filter data ####
data_segments = pd.read_csv('/content/drive/Shareddrives/Neuromancers_Data/EEG_Data_Segmentation.csv')

filtered_data = {"nabeha":[], "heidi":[]}

# Iterate thru all trials for each subject
for subj in filtered_data.keys():
  for i in range(5):
    # Filter current trial data
    curr_trial = raw_data[subj][i]
    print(curr_trial)
    filtered_trial = curr_trial.copy().filter(l_freq=0.5, h_freq=45, picks=None, 
                                 method='fir', fir_design='firwin', 
                                 l_trans_bandwidth='auto', h_trans_bandwidth='auto', 
                                 filter_length='auto', phase='zero')
    filtered_trial = filtered_trial.filter(l_freq=0.5, h_freq=45, picks=None, 
                                 method='iir', l_trans_bandwidth='auto',
                                 h_trans_bandwidth='auto', filter_length='auto', phase='zero')
    
    # Crop filtered_trial to within experiment duration
    curr_row = data_segments[data_segments["Participant"] == subj+'_'+str(i+1)].index
    start = data_segments.at[curr_row[0], "Experiment Start"]
    end = data_segments.at[curr_row[0], "Experiment End"]
    print(f"{subj}_{str(i+1)}: {start} {end}")
    filtered_trial = filtered_trial.crop(tmin=start,tmax=end)
    
    # Add filtered_trial to filtered_data dictionary
    filtered_data[subj].append(filtered_trial)

In [ ]:
#### Check for bad channels ####
for i in range(5):
  print(f"-------- Trial {i+1} --------")
  trial = filtered_data["nabeha"][i]
  trial.plot(duration=20)
  trial.plot_psd(fmax=45)